In [42]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# modelling:
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [43]:
df = pd.read_csv('data/stud.csv')

In [76]:

df.head()
edu_map = {
    "some high school":0,
    "high school":1,
    "some college":2,
    "associate's degree":3,
    "bachelor's degree":4,
    "master's degree":5
}

df["parent_edu_rank"] = df["parental_level_of_education"].map(edu_map)
df["prep_effect"] = (df["test_preparation_course"] == "completed").astype(int) * df["reading_score"]

df["lunch_effect"] = (df["lunch"] == "standard").astype(int) * df["writing_score"]

# Prepare X and Y variables:

In [77]:
X=df.drop("math_score",axis=1)
y=df["math_score"]
X.head()
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [80]:
cat_features=df.select_dtypes(exclude="number")
num_features=df.select_dtypes(exclude="object")
cat_features,num_features

(     gender race_ethnicity parental_level_of_education         lunch  \
 0    female        group B           bachelor's degree      standard   
 1    female        group C                some college      standard   
 2    female        group B             master's degree      standard   
 3      male        group A          associate's degree  free/reduced   
 4      male        group C                some college      standard   
 ..      ...            ...                         ...           ...   
 995  female        group E             master's degree      standard   
 996    male        group C                 high school  free/reduced   
 997  female        group C                 high school  free/reduced   
 998  female        group D                some college      standard   
 999  female        group D                some college  free/reduced   
 
     test_preparation_course  
 0                      none  
 1                 completed  
 2                      none 

In [81]:
for col in cat_features:
  print(df[col].value_counts())

gender
female    518
male      482
Name: count, dtype: int64
race_ethnicity
group C    319
group D    262
group B    190
group E    140
group A     89
Name: count, dtype: int64
parental_level_of_education
some college          226
associate's degree    222
high school           196
some high school      179
bachelor's degree     118
master's degree        59
Name: count, dtype: int64
lunch
standard        645
free/reduced    355
Name: count, dtype: int64
test_preparation_course
none         642
completed    358
Name: count, dtype: int64


In [82]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [83]:
X.shape

(1000, 10)

In [84]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 10), (200, 10))

In [85]:
def evaluate_model(true,predicted):
  mae=mean_absolute_error(true,predicted)
  mse=mean_absolute_error(true,predicted)
  rmse=np.sqrt(mean_squared_error(true,predicted))
  r2_square=r2_score(true,predicted)
  return mae,rmse,r2_square

In [86]:
from sklearn.pipeline import Pipeline

def make_pipeline(model):
    return Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])

models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "KNN": KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(),
    "XGBoost": XGBRegressor(),
    "CatBoost": CatBoostRegressor(verbose=0),
    "AdaBoost": AdaBoostRegressor()
}


In [87]:
from sklearn.model_selection import cross_validate
import pandas as pd

results = []

for name, model in models.items():
    
    pipe = make_pipeline(model)

    scores = cross_validate(
        pipe,
        X,
        y,
        cv=5,
        scoring=("r2","neg_mean_absolute_error","neg_root_mean_squared_error"),
        n_jobs=-1
    )

    results.append({
        "Model": name,
        "R2": scores["test_r2"].mean(),
        "MAE": -scores["test_neg_mean_absolute_error"].mean(),
        "RMSE": -scores["test_neg_root_mean_squared_error"].mean()
    })

In [89]:
results_df = pd.DataFrame(results).sort_values(by="R2", ascending=False)
print(results_df)

              Model        R2       MAE      RMSE
2             Ridge  0.871627  4.298714  5.399786
0  LinearRegression  0.871552  4.299888  5.401370
7          CatBoost  0.845638  4.710291  5.921995
5      RandomForest  0.838877  4.796811  6.057059
8          AdaBoost  0.829848  4.941978  6.222368
1             Lasso  0.819739  5.054770  6.400165
6           XGBoost  0.813624  5.150597  6.513748
3               KNN  0.742796  5.994800  7.643583
4      DecisionTree  0.709868  6.390000  8.130567


In [90]:
# XGBoost tuning
xgb_pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("model", XGBRegressor(objective="reg:squarederror", random_state=42))
])

In [91]:
param_dist = {
    "model__n_estimators": [200, 400, 600, 800],
    "model__learning_rate": [0.01, 0.03, 0.05, 0.1],
    "model__max_depth": [3, 4, 5, 6, 8],
    "model__subsample": [0.6, 0.7, 0.8, 1.0],
    "model__colsample_bytree": [0.6, 0.7, 0.8, 1.0],
    "model__gamma": [0, 0.1, 0.25, 0.5],
    "model__reg_lambda": [1, 1.5, 2, 3]
}
search = RandomizedSearchCV(
    xgb_pipe,
    param_distributions=param_dist,
    n_iter=30,          # try 30 random combinations
    cv=5,
    scoring="r2",
    verbose=2,
    n_jobs=-1,
    random_state=42
)

search.fit(X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","Pipeline(step...=None, ...))])"
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'model__colsample_bytree': [0.6, 0.7, ...], 'model__gamma': [0, 0.1, ...], 'model__learning_rate': [0.01, 0.03, ...], 'model__max_depth': [3, 4, ...], ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",30
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'r2'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies 

In [92]:
print("Best Parameters:")
print(search.best_params_)

print("\nBest CV R2 Score:")
print(search.best_score_)

Best Parameters:
{'model__subsample': 0.8, 'model__reg_lambda': 1, 'model__n_estimators': 200, 'model__max_depth': 3, 'model__learning_rate': 0.05, 'model__gamma': 0.25, 'model__colsample_bytree': 0.7}

Best CV R2 Score:
0.8582968831062316


In [93]:
best_model = search.best_estimator_

y_pred = best_model.predict(X_test)

In [94]:
# ridge tuning:

X["literacy_score"] = (X["reading_score"] + X["writing_score"]) / 2
X["read_write_gap"] = abs(X["reading_score"] - X["writing_score"])


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape


from sklearn.model_selection import GridSearchCV


ridge_pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("model", Ridge())
])

params = {
    "model__alpha": [0.01,0.1,0.5,1,2,5,10,20,50,100]
}

search = GridSearchCV(ridge_pipe, params, cv=5, scoring="r2")
search.fit(X, y)

print(search.best_params_)
print(search.best_score_)

{'model__alpha': 2}
0.8716502234814012
